# Exploratory Data Analisis (EDA)
En este notebook me enfoco en:
- Recolección de datos desde distintas fuentes
- Identificado diversas estructuras de los datasets
- Análisis de nulos, duplicados, relaciones
- Checklist de problemas identificados

## 1. Configuración inicial
- Importación de librerias

In [40]:
# Importado de librerías
import pandas as pd
import os
import json
import subprocess
import tabula

## 2. Obtención de datasets

- Obtención del dataset de ventas y paso a DataFrame

In [41]:
# Cargar credenciales de Kaggle desde un archivo JSON
with open("kaggle_keys.json") as f:
    kaggle_creds = json.load(f)

os.environ["KAGGLE_USERNAME"] = kaggle_creds["username"]
os.environ["KAGGLE_KEY"] = kaggle_creds["key"]

dataset = "virtualschool/restaurant-cost-and-sales-dataset"

# Ejecuta el comando kaggle datasets download
subprocess.run([
    "kaggle", "datasets", "download",
    "-d", dataset,
    "--unzip",
    "-p", 'data/raw'
])

dir_raw = r"data\raw"
excel_restaurante = dir_raw + r"\Restaurant_Data.xlsx"

df_orders = pd.read_excel(excel_restaurante, sheet_name="Orders")
df_items = pd.read_excel(excel_restaurante, sheet_name="Items")

- Dataset del INDEC

In [42]:
path_informe = dir_raw + r"\INFORME_INDEC_1-10-25.pdf"

# Obtener DataFrame que me interesa
df_indec = tabula.read_pdf(
    path_informe,
    pages=12,
    multiple_tables=True,
    encoding='latin1'  
)[0]

- Valor del dólar

In [43]:
from utils import realizar_request

url_dolarapi = 'https://dolarapi.com/v1/dolares/blue'

request_ok, data = realizar_request(url_dolarapi)
if not request_ok:
    raise RuntimeError("No se pudo obtener el valor del dólar desde la API.")
dolar_blue = float(data.get("venta"))

## 3. Analizado de DataFrames y detección de tareas
- Vista general

In [44]:
print(df_orders.info())
print('------------------')
print(df_items.info())
print('------------------')
print(df_indec.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73000 entries, 0 to 72999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          73000 non-null  datetime64[ns]
 1   Time          73000 non-null  object        
 2   Order Number  73000 non-null  int64         
 3   Item          73000 non-null  int64         
 4   Count         73000 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 2.8+ MB
None
------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Item          71 non-null     int64  
 1   Category      71 non-null     object 
 2   Sub Category  68 non-null     object 
 3   Item Name     68 non-null     object 
 4   Price         71 non-null     int64  
 5   Cost          71 non-null     float64
dty

> Dado a lo que pareciera ser un problema en las columnas del INDEC, realizo una vista más detallada

In [48]:
print(df_indec.head())

  Unnamed: 0 Unnamed: 1 Unnamed: 2     Precios  Unnamed: 3        Variación
0        NaN  Unidad de        NaN         NaN         NaN       porcentual
1   Variedad        NaN        NaN         NaN         NaN              NaN
2        NaN     medida    Octubre  Septiembre         NaN  respecto al mes
3        NaN        NaN    de 2025     de 2025         NaN         anterior
4        NaN        NaN        NaN       Pesos         NaN                %


> Los nombres de las columnas parecieran no estar perfectamente alineados con el encabezado del DataFrame, pero están presentes!

- Ahora, **detección de nulos y duplicados**:

In [46]:
print("Cantidad de filas y columnas - ", "Órdenes: ", df_orders.shape, "- Items: ", df_items.shape,"- INDEC: ", df_indec.shape)
print("-------------------")
print("Cantidad de valores nulos - ", "Órdenes: ", df_orders.isnull().sum().sum(), "- Items: ", df_items.isnull().sum().sum(),"- INDEC: ", df_indec.isnull().sum().sum())
print("-------------------")
print("Cantidad de duplicados - ", "Órdenes: ", df_orders.duplicated().sum(), "- Items: ", df_items.duplicated().sum(),"- INDEC: ", df_indec.duplicated().sum())

Cantidad de filas y columnas -  Órdenes:  (73000, 5) - Items:  (71, 6) - INDEC:  (64, 6)
-------------------
Cantidad de valores nulos -  Órdenes:  0 - Items:  6 - INDEC:  77
-------------------
Cantidad de duplicados -  Órdenes:  0 - Items:  0 - INDEC:  0


## 4. Pasado a formato CSV para la posterior realización de tareas

In [47]:
staging_dir = r"data\staging"

# Almacenado de los DataFrames en formato CSV
df_orders.to_csv(os.path.join(staging_dir, "orders.csv"), index=False)
df_items.to_csv(os.path.join(staging_dir, "items.csv"), index=False)
df_indec.to_csv(os.path.join(staging_dir, "indec.csv"), index=False)

# Guardo el valor del dólar blue en un archivo de texto
with open(os.path.join(staging_dir, "dolar_blue.txt"), "w") as f:
    f.write(str(dolar_blue))


## 5. Síntesis del Exploratory Data Analysis

**Se puede apreciar fácilmente:**
- Diferencias de lenguaje: INDEC en español, pero ventas en inglés
- Diferencias entre formatos (preferible pasarlo todo a `snake_case`)
- Valores nulos varios: 77 en el INDEC y 6 en Items
- Formateo necesario del DataFrame del INDEC con asignación de columnas

**A realizar:**
- Asignación de columnas que me interesan del INDEC (`Variedad`, `Unidad de medida` y `Octubre de 2025`)
- Limpieza de valores nulos
- Traducción al español del dataset de Kaggle
- Formateo a `snake_case` y eliminación de tildes
